In [1]:
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np
import os

hv.extension('bokeh')

## Select the reservoir

In [2]:
RESERVOIR = '0214'

In [3]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/glws-tmsos-baseline-comparison-locations-pts.geojson'))
val_polys = gpd.read_file(Path('/tiger1/pdas47/tmsosPP/data/validation-locations/glws-tmsos-baseline-comparison-locations-polys.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()  # select all 100 reservoirs
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name'] # dictionary that can be queried to get reservoir name

RESERVOIR_NAME = res_names[RESERVOIR]

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

# get reservoir properties from GRanD
reservoir = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]
nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area

min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area

area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])


## Plot a map of the selected reservoirs
global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

print(
    f"Selected reservoir: {RESERVOIR}: {RESERVOIR_NAME}\n",
    f"{nominal_area = }\n",
    f"{nominal_area_poly = }\n",
    f"{max_area = }\n",
    f"{min_area = }\n",
    f"{area_rep = }\n",
    f"{dam_height = }\n",
    f"{elev_msl = }\n",
    f"{depth = }\n",
    f"{capacity = }\n",
)

global_map

Selected reservoir: 0214: Cijara Dam, Sp
 nominal_area = 30.22
 nominal_area_poly = 30.22
 max_area = 66.0
 min_area = 0
 area_rep = 65.6
 dam_height = 81.0
 elev_msl = 407.0
 depth = 54.3
 capacity = 1640.0



:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

In [4]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [5]:
srtm_extrapolated_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/aev/')

In [6]:
import numpy as np


if dam_height == -99:
    dam_height = np.nan
if elev_msl == -99:
    elev_msl = np.nan

In [7]:
import ee
ee.Initialize()

In [8]:
lat = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['LAT_DD'].values[0]
lon = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['LONG_DD'].values[0]

In [9]:
val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]

AttributeError: 'NoneType' object has no attribute 'copy'

,tmsos_id,TMS-OS,GLWS (Poly),GLWS (LinearLi),GLWS (SimpleLinear),GRAND_ID,RES_NAME,DAM_NAME,ALT_NAME,RIVER,...,name,rid_id,grand_id,rid_filepath,resops_id,rid_filename,distance,COUNTRY_SHORT,left_only,geometry
2,0214,NaN,NaN,NaN,1.0,2811,None,Cijara,None,Guadiana,...,"Cijara Dam, Sp",NaN,NaN,None,NaN,None,0.000406,Sp,both,"POLYGON ((-4.96212 39.42249, -4.96139 39.42138..."


In [10]:
def get_obs_aec_above_water_surface(aec, max_height):
    """
    Filters and processes the AEC (Area-Elevation Curve) data to obtain observations above a specified water surface height.

    Args:
        aec (pd.DataFrame): DataFrame containing AEC data with 'Elevation' and 'CumArea' columns.
        max_height (float): The maximum height of the water surface to filter the observations.

    Returns:
        pd.DataFrame: A DataFrame containing the filtered and processed AEC data with 'Elevation' and 'CumArea' columns.
    """
    obs_aec_above_water = aec[aec['Elevation'] < max_height]
    obs_aec_above_water = obs_aec_above_water.sort_values('Elevation')
    obs_aec_above_water['CumArea_diff'] = obs_aec_above_water['CumArea'].diff()
    obs_aec_above_water['z_score'] = (obs_aec_above_water['CumArea_diff'] - obs_aec_above_water['CumArea_diff'].mean()) / obs_aec_above_water['CumArea_diff'].std()
    max_z_core_idx = obs_aec_above_water['z_score'].idxmax()
    print(f"max_z_core_idx: {max_z_core_idx}")
    obs_aec_above_water = obs_aec_above_water.loc[max_z_core_idx:, :]
    obs_aec_above_water = obs_aec_above_water[['Elevation', 'CumArea', 'z_score']]

    return obs_aec_above_water


In [11]:
from scipy.integrate import cumulative_trapezoid

def calculate_storage(aec_df):
    """
    Calculate the storage of a reservoir from its Area-Elevation Curve (AEC).

    Parameters:
    aec_df (pd.DataFrame): DataFrame containing 'Elevation' and 'CumArea' columns.

    Returns:
    pd.DataFrame: DataFrame with an additional 'Storage' column representing the storage in cubic meters.
    """
    elevation_normalized = (aec_df['Elevation'] - aec_df['Elevation'].min())

    # cumulative_trapezoid takes two parameters.
    # y = y-axis locations of points. these values will be integrated. 
    # x = x-axis locations of points, where each y value is sampled. Area.
    storage = cumulative_trapezoid(
        elevation_normalized, 
        aec_df['CumArea'] * 1e6
    )
    storage = np.insert(storage, 0, 0)

    aec_df['Storage'] = storage
    aec_df['Storage (mil. m3)'] = storage * 1e-6
    return aec_df

In [12]:
from scipy.optimize import minimize

# Function to return predicted y-values from the polynomial
def predict_y(params, x):
    return np.polyval(params, x)

# Objective function: residuals to minimize
def objective(params, x, y):
    predicted_y = predict_y(params, x)

    return np.sum((predicted_y - y)**2)

# Constraint 1: dy/dx > 0 -> derivative of the polynomial should be positive for all x
def constraint_derivative(params, x):
    # Sample points across the entire range of x (0 to np.max(x))
    x_sample = np.linspace(0, np.max(x), 100)
    
    # Get the derivative coefficients of the polynomial
    derivative_coeffs = np.polyder(params[::-1], 1)  # Take the first derivative
    
    # Evaluate the derivative at these sample points
    derivative_values = np.polyval(derivative_coeffs, x_sample)  # np.polyder gives the derivative coefficients

    # Return the minimum value of the derivative; it should be greater than 0
    return derivative_values

# Constraint 2: intercept should be within (dam_bottom, dam_bottom + 5)
def constraint_intercept(params, dam_bottom):
    a0 = params[-1]  # Intercept is the last parameter (highest degree)
    return a0 - dam_bottom, (dam_bottom + 5) - a0

# Function to perform the minimization
def fit_polynomial(x, y, degree, dam_bottom):
    # Initial guess for the polynomial parameters using np.polyfit
    initial_guess = np.polyfit(x, y, degree)
    print("Initial guess: ", initial_guess[::-1])
    
    # Set up constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda params: constraint_derivative(params, x)},  # dy/dx > 0
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params, dam_bottom)[0]},  # intercept within lower bound
        {'type': 'ineq', 'fun': lambda params: constraint_intercept(params, dam_bottom)[1]}   # intercept within upper bound
    ]
    
    # Perform minimization
    result = minimize(objective, initial_guess, args=(x, y), constraints=constraints, options={'maxiter': 1000})
    
    # Print results
    print(f"Optimized polynomial for degree {degree}: {np.poly1d(result.x[::-1])}")
    return result, initial_guess

In [13]:
from pathlib import Path
from itertools import zip_longest,chain
from rat.ee_utils.ee_utils import poly2feature

BUFFER_DIST = 500
DEM = ee.Image('USGS/SRTMGL1_003')

def grouper(iterable, n, *, incomplete='fill', fillvalue=None):
    "Collect data into non-overlapping fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, fillvalue='x') --> ABC DEF Gxx
    # grouper('ABCDEFG', 3, incomplete='strict') --> ABC DEF ValueError
    # grouper('ABCDEFG', 3, incomplete='ignore') --> ABC DEF
    args = [iter(iterable)] * n
    if incomplete == 'fill':
        return zip_longest(*args, fillvalue=fillvalue)
    if incomplete == 'strict':
        return zip(*args, strict=True)
    if incomplete == 'ignore':
        return zip(*args)
    else:
        raise ValueError('Expected fill, strict, or ignore')

def _aec(n,elev_dem,roi, scale=30):
    ii = ee.Image.constant(n).reproject(elev_dem.projection())
    DEM141 = elev_dem.lte(ii)

    DEM141Count = DEM141.reduceRegion(
        geometry= roi,
        scale= scale,
        reducer= ee.Reducer.sum()
    )
    area=ee.Number(DEM141Count.get('elevation')).multiply(30*30).divide(1e6)
    return area

def get_obs_aec_srtm(aec_dir_path, scale, reservoir, reservoir_name, force=False):
    """
    Generates an observed Area-Elevation Curve (AEC) file for a given reservoir using SRTM data.
    The function checks if an AEC file already exists for the given reservoir. If it does, it reads the file and returns the data.
    If not, it generates the AEC using SRTM data. It saves the data as a csv file and returns the aecas a pandas dataframe.
    Parameters:
        aec_dir_path (str): The directory path where the AEC file will be saved.
        scale (int): The scale at which to perform the elevation calculations.
        reservoir (object): The reservoir object containing geometry information.
        reservoir_name (str): The name of the reservoir.
        force (bool): If True, the function will generate the AEC file even if it already exists.
    Returns:
        pd.DataFrame: A DataFrame containing the elevation and cumulative area data.
    """
    print(f"Generating observed AEC file for {reservoir_name} from SRTM")
    aec_dst_fp = os.path.join(aec_dir_path,reservoir_name+'.csv')
    print(aec_dst_fp)

    if Path(aec_dst_fp).exists() and not force:
        print(f"SRTM AEC exists for {reservoir_name} at {aec_dst_fp}")
        aec_df = pd.read_csv(aec_dst_fp, comment='#')
    else:
        reservoir_polygon = reservoir.geometry
        aoi = poly2feature(reservoir_polygon,BUFFER_DIST).geometry()
        min_elev = DEM.reduceRegion( reducer = ee.Reducer.min(),
                            geometry = aoi,
                            scale = scale,
                            maxPixels = 1e16,
                            bestEffort=True
                            ).get('elevation')
        max_elev = DEM.reduceRegion( reducer = ee.Reducer.max(),
                            geometry = aoi,
                            scale = scale,
                            maxPixels = 1e16,
                            bestEffort=True
                            ).get('elevation')

        elevs = ee.List.sequence(min_elev, max_elev, 1)
        elevs_list = elevs.getInfo()
        grouped_elevs_list = grouper(elevs_list,5)

        areas_list = []
        for subset_elevs_tuples in grouped_elevs_list:
                ## Removing None objects from grouped tuples and converting them to list and then ee.List and then calculating  area for that 
            subset_elevs_list = list(filter(lambda x: x != None, subset_elevs_tuples))
            subset_elevs = ee.List(subset_elevs_list)
            subset_areas = subset_elevs.map(lambda elevation_i: _aec(elevation_i, DEM, aoi, scale))
            subset_areas_list = subset_areas.getInfo()
                    ## Appendning areas to the main list
            areas_list.append(subset_areas_list)
                    
                ## Converting list of lists to list
        areas_list = list(chain(*areas_list))
        areas_list = np.round(areas_list,3)
                ## Preparing dataframe to write down to csv 
        aec_df = pd.DataFrame(
                    data = {
                        'Elevation' :elevs_list, 
                        'CumArea':areas_list
                        })
        aec_df.to_csv(aec_dst_fp,index=False)
        print(f"Observed AEC obtained from SRTM for {reservoir_name}")

    return aec_df

In [22]:
from shapely.geometry import Point

def dam_bottom_dem_percentile(
        dam_location, scale=30,
        percentiles=[1, 2, 3, 4, 5, 10], buffer_distance=1000,
        ee_dem_name = 'MERIT/DEM/v1_0_3'
    ):
    """
    Calculates elevations corresponding to percentile values within a buffered region around 
    a dam location.

    Parameters:
        dam_location (shapely.geometry.point.Point): The location of the dam as a point geometry.
        scale (int, optional): Scale in meters for the DEM data. Default is 30.
        percentiles (list of int, optional): List of percentiles to calculate. Default is [1, 2, 3, 4, 5, 10].
        buffer_distance (int, optional): Buffer distance in meters around the point of interest. Default is 1000.
        ee_dem_name (str, optional): Name of the Earth Engine DEM dataset. Default is 'MERIT/DEM/v1_0_3'.
    Returns:
        dict: A dictionary containing the calculated statistics (min, max, and specified percentiles).
    """
    lon, lat = dam_location.x, dam_location.y
    # Create a point geometry using the provided latitude and longitude
    point = ee.Geometry.Point([lon, lat])
    
    # Buffer the point to create a region of interest (ROI)
    feature = point.buffer(buffer_distance)
    # Load the DEM dataset
    dem = ee.Image(ee_dem_name)
    
    # Clip DEM to the buffered region (feature)
    dem_clipped = dem.clip(feature)
    
    # Define the reducer (min/max combined with percentiles)
    reducer = (ee.Reducer.minMax()
               .combine(ee.Reducer.percentile(percentiles), '', True))
    
    # Reduce the clipped DEM over the feature's geometry to get statistics
    stats = dem_clipped.reduceRegion(
        reducer=reducer,
        geometry=feature,
        scale=scale,  # Scale appropriate for MERIT DEM (30 meters)
        maxPixels=1e9
    )
    
    # Set the calculated statistics as properties of the feature
    return stats.getInfo()

def get_closest_point_to_dam(
        reservoir, dam_location, grwl_fp, buffer_distance=90
    ):
    """
    Gets the closest point to the dam for a given reservoir.

    Parameters:
        reservoir (gpd.GeoSeries): GeoSeries containing the reservoir geometry.
        dam_location (shapely.geometry.point.Point): The location of the dam as a point geometry.
        buffer_distance (int): The buffer distance around the reservoir in meters.
        grwl_fp (Path): The file path to the GRWL data.

    Returns:
        shapely.geometry.point.Point: The closest point to the dam.
    """
    grwl = gpd.read_file(grwl_fp)

    # Select the reservoir
    reservoir_df = reservoir

    # Clip GRWL by taking a 1 degree buffer around reservoir_df
    reservoir_df_buffered = reservoir_df.buffer(1)
    grwl_clipped = gpd.clip(grwl, reservoir_df_buffered)

    # Convert the CRS to the local UTM projection
    utm_crs = reservoir_df.estimate_utm_crs()
    grwl_utm = grwl_clipped.to_crs(utm_crs)
    reservoir_utm = reservoir_df.to_crs(utm_crs)

    # Take a buffer of buffer_distance meters around the grand reservoirs
    reservoir_utm_buffered = reservoir_utm.geometry.buffer(buffer_distance)

    # Convert buffered geometries back to GeoDataFrame
    reservoir_utm_buffered_gdf = gpd.GeoDataFrame(geometry=reservoir_utm_buffered, crs=utm_crs).reset_index(drop=True)

    # Reverse the clipping to keep the portion of GRWL polylines outside the reservoir polygon
    grwl_outside_reservoir = gpd.overlay(grwl_utm, reservoir_utm_buffered_gdf, how='difference')

    # Get the dam location in utm crs
    dam_location_geom = gpd.GeoDataFrame(geometry=[dam_location], crs=reservoir_df.crs).to_crs(utm_crs)

    # Convert the dam location and grwl_outside_reservoir back to lat/lon
    dam_location_latlon = dam_location_geom.to_crs(epsg=4326).geometry.iloc[0]
    grwl_outside_reservoir_latlon = grwl_outside_reservoir.to_crs(epsg=4326)

    # Find the closest point in the grwl_outside_reservoir polylines to the dam location
    closest_point_to_dam = grwl_outside_reservoir_latlon.geometry.apply(lambda geom: geom.interpolate(geom.project(dam_location_latlon)))
    closest_point_to_dam = closest_point_to_dam.iloc[closest_point_to_dam.distance(dam_location_latlon).idxmin()]

    return closest_point_to_dam


def get_dam_bottom(
        reservoir, 
        dam_location=None, 
        grwl_fp=None,
        buffer_distance=500, 
        grwl_intersection_buffer=250,
        grwl_buffer_from_dam=90,
        
    ):
    """
    Determines the dam bottom elevation for a given reservoir. It can either use centerlines of
    rivers (GRWL) to determine the likely location the downstream river bed, or use a buffered
    region around the dam location to find the 1 percentile elevation as an estimate of the 
    elevation of the dam bottom.

    Parameters:
        reservoir (gpd.GeoSeries): GeoSeries containing the reservoir geometry.
        dam_location (shapely.geometry.point.Point): The location of the dam as a point geometry.
        grwl_fp (Path): The file path to the GRWL data. If passed, GRWL river centerlines would
            be used to estimate the location of the dam bottom.  
        buffer_distance (int): Buffer distance around the dam in meters to use if the dam location
            is to be used for estimating the dam bottom.
        grwl_intersection_buffer (float): Buffer distance of a region to consider around the
            intersection of GRWL and reservoir boundary for finding the bottom-elevation from the
            distribution of elevations within that region. Defaults to 250 m.
        grwl_buffer_from_dam (float): Buffer distance to use along the GRWL centerline from the
            intersection of GRWL and reservoir boundary for estimating the location of the 
            dam bottom.

    Returns:
        tuple: (float, str)
            The dam bottom elevation and the method used ('grwl_intersection' or 'dam_location').
    """
    if grwl_fp is not None:
        # Load GRWL data
        grwl = gpd.read_file(grwl_fp)
        # Check if GRWL intersects with reservoir geometry
        intersection = grwl.intersects(reservoir.union_all())
    else:
        intersection = np.array([False])

    if intersection.any():
        method = 'grwl_intersection'
        print("GRWL intersects with reservoir geometry.")
        closest_point_to_dam = get_closest_point_to_dam(
            reservoir, dam_location,
            grwl_fp=grwl_fp,
            buffer_distance=grwl_buffer_from_dam,  # 90 m buffer from reservoir = 3 pixels
        )
        
        # Extract latitude and longitude of the closest point
        lat, lon = closest_point_to_dam.y, closest_point_to_dam.x
        
        # Calculate the dam bottom elevation using the dam_bottom_dem_percentile function
        stats = dam_bottom_dem_percentile(closest_point_to_dam, buffer_distance=grwl_intersection_buffer)
        
        # Extract the 1 percentile elevation as the dam bottom elevation
        dam_bottom_elevation = stats['dem_p1']
    else:
        method = 'dam_location'
        if grwl_fp is None:
            print("GRWL dataset is not provided. You can download global_data which includes GRWL.")
        else:
            print("GRWL does not intersect with reservoir geometry.")
        
        # Calculate the dam bottom elevation using the dam_bottom_dem_percentile function with a 500 m buffer
        stats = dam_bottom_dem_percentile(dam_location, buffer_distance=buffer_distance)
        
        # Extract the 1 percentile elevation as the dam bottom elevation
        dam_bottom_elevation = stats['dem_p1']

    return dam_bottom_elevation, method



reservoir = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]
# Extract latitude and longitude for the reservoir
lat_dd = reservoir['LAT_DD'].values[0]
long_dd = reservoir['LONG_DD'].values[0]

# Construct dam_location using the extracted latitude and longitude
dam_location = Point(long_dd, lat_dd)

get_dam_bottom(
    reservoir, dam_location, grwl_fp=Path('/tiger1/pdas47/tmsosPP/data/dam_bottom_elevation/GRWL/GRWL_summaryStats.shp')
)

GRWL intersects with reservoir geometry.


/tmp/ipykernel_861101/3802064751.py:69: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reservoir_df_buffered = reservoir_df.buffer(1)
/tmp/ipykernel_861101/3802064751.py:95: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  closest_point_to_dam = closest_point_to_dam.iloc[closest_point_to_dam.distance(dam_location_latlon).idxmin()]


(407.8217168016446, 'grwl_intersection')

In [23]:
def extrapolate_reservoir(
    reservoir, dam_location, reservoir_name, dam_height, aec, aev_save_dir, buffer_distance=500,
    grwl_fp=None
):
    """
    Extrapolates the reservoir's Area-Elevation-Capacity (AEC) curve by fitting a polynomial to observed data and adds a column for storage.
    - The function first calculates the dam bottom elevation using the specified percentile or GRWL intersection.
    - It then fits polynomials of degrees 2 and 1 to the observed AEC data, stopping at the first successful fit.
    - The predicted AEC curve is generated and saved to a CSV file in the specified directory.
    - The initial guess and polynomial equation are stored as comments in the CSV file.
    
    Parameters:
        reservoir (gpd.GeoSeries): GeoSeries containing the reservoir geometry.
        dam_location (shapely.geometry.point.Point): The location of the dam as a point geometry.
        reservoir_name (str): Name of the reservoir.
        dam_height (float): Height of the dam.
        aec (pd.DataFrame): DataFrame containing observed AEC data with columns 'CumArea' and 'Elevation'.
        aev_save_dir (str): Directory path to save the predicted AEC file.
        buffer_distance (int, optional): Buffer distance around the dam. Default is 500. It will be used if GRWL
            doesn't intersect with the reservoir geometry or if not provided.
        grwl_fp (Path, optional): File path to the GRWL (Global River Widths from Landsat) dataset. Default is None.
    
    Returns:
        pd.DataFrame: DataFrame containing the predicted storage values with columns 'CumArea', 'Elevation', 'Storage', 'Storage (mil. m3)', and 'Elevation_Observed'.
    """
    dam_bottom_elevation, method = get_dam_bottom(reservoir, buffer_distance=buffer_distance, dam_location=dam_location, grwl_fp=grwl_fp) # from GRWL downstream point
    dam_top_elevation = dam_bottom_elevation + dam_height

    print(f"Dam bottom elevation for {reservoir_name} is {dam_bottom_elevation}")
    print(f"Dam top elevation for {reservoir_name} is {dam_top_elevation}")

    aec = aec[(aec['Elevation'] > dam_bottom_elevation)&(aec['Elevation'] <= dam_top_elevation)]
    x = aec['CumArea']
    y = aec['Elevation']

    # Try fitting polynomials starting from degree 3 down to 1
    for degree in [2, 1]:
        result, initial_guess = fit_polynomial(x, y, degree, dam_bottom_elevation)
        if result.success:
            print(f"Successfully fitted a degree {degree} polynomial for {reservoir_name}")
            break
        else:
            print(f"Failed to fit a degree {degree} polynomial for {reservoir_name}, trying lower degree")
    
    # Generate x_pred by combining a range of values from 0 to the maximum of x with the unique values from aec['CumArea']
    # and then sorting them. This ensures that x_pred includes all unique cumulative area values from the AEC data.
    # This was we can also store the observed SRTM AEC values.
    x_pred = np.arange(0, np.max(x), 0.25)
    x_pred = np.unique(np.concatenate((x_pred, aec['CumArea'].values)))
    x_pred = np.sort(x_pred)
    y_pred = predict_y(result.x, x_pred)

    # Create a new DataFrame with the predicted x and y values
    predicted_df = pd.DataFrame({
        'CumArea': x_pred, 
        'Elevation': y_pred
    })

    predicted_storage_df = calculate_storage(predicted_df)
    # Merge the predicted DataFrame with the observed AEC DataFrame to get the observed elevations
    predicted_storage_df = pd.merge(
        predicted_storage_df, 
        aec[['CumArea', 'Elevation']], 
        on='CumArea', 
        how='left', 
        suffixes=('', '_Observed')
    )
    # Store the initial guess and result as comments in the AEC file
    with open(os.path.join(aev_save_dir, f"{reservoir_name}.csv"), 'w') as f:
        f.write(f"# Initial guess: {initial_guess}\n")
        f.write(f"# Polynomial degree: {degree}\n")
        polynomial_equation = np.poly1d(result.x)
        for line in str(polynomial_equation).split('\n'):
            f.write(f"# {line}\n")
        f.write(f"# dam bottom method: {method}\n")
        predicted_storage_df.to_csv(f, index=False)
    print(f"Saved predicted AEC for {reservoir_name} to {os.path.join(aev_save_dir, f'{reservoir_name}.csv')}")

    return predicted_storage_df


In [24]:
reservoir_name = RESERVOIR_NAME
dam_height = reservoir['DAM_HGT_M'].values[0]
custom_dam_heights = {}

if dam_height == -99:
    dam_height = custom_dam_heights.get(RESERVOIR)

# aec = get_obs_aec_srtm(
#     '/tiger1/pdas47/tmsosPP/data/aec/srtm', 30, 
#     reservoir.squeeze(), 
#     RESERVOIR
# )

In [25]:
from shapely.geometry import Point

aev_save_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/aev_extrapolated')

# Extract latitude and longitude for the reservoir
lat_dd = reservoir['LAT_DD'].values[0]
long_dd = reservoir['LONG_DD'].values[0]

srtm_aec = get_obs_aec_srtm(
    srtm_extrapolated_dir, 30, reservoir.squeeze(), RESERVOIR, force=False
)

# Create dam_location using the extracted latitude and longitude
dam_location = Point(long_dd, lat_dd)
# aec_files = Path("/tiger1/pdas47/tmsosPP/data/aec/aev").rglob("*.csv")
# aec_fn = Path(f"/tiger1/pdas47/tmsosPP/data/aec/srtm_rat/{RESERVOIR}.csv")
extrapolated_aec = extrapolate_reservoir(
    reservoir=reservoir, dam_location=dam_location, 
    reservoir_name=RESERVOIR, dam_height=dam_height, aec=srtm_aec, aev_save_dir=aev_save_dir
)

Generating observed AEC file for 0214 from SRTM
/tiger1/pdas47/tmsosPP/data/aec/aev/0214.csv
SRTM AEC exists for 0214 at /tiger1/pdas47/tmsosPP/data/aec/aev/0214.csv
GRWL dataset is not provided. You can download global_data which includes GRWL.
Dam bottom elevation for 0214 is 389.0168151855469
Dam top elevation for 0214 is 470.0168151855469
Initial guess:  [ 3.98191622e+02 -1.74273062e-02  5.87446144e-03]
Optimized polynomial for degree 2:      2
394 x + 0.09588 x + 0.005157
Successfully fitted a degree 2 polynomial for 0214
Saved predicted AEC for 0214 to /tiger1/pdas47/tmsosPP/data/aec/aev_extrapolated/0214.csv


In [ ]:
# srtm_aec['z_score'] = (srtm_aec['CumArea'].diff() - srtm_aec['CumArea'].diff().mean()) / srtm_aec['CumArea'].diff().std()
# srtm_aec['z_score'] = srtm_aec['z_score'].shift(-1)
# srtm_aec['z_score'] = srtm_aec['z_score'].fillna(0)

# srtm_aec.hvplot(
#     y='z_score', x='Elevation', kind='scatter', title=f"Z-Score of AEC for {RESERVOIR_NAME}"
# )

:Scatter   [Elevation]   (z_score)

In [ ]:
# zscore_dir = Path('/tiger1/pdas47/tmsosPP/data/aec/zscore')
# srtm_aec.to_csv(zscore_dir / f"{RESERVOIR}-{srtm_aec.loc[srtm_aec['z_score'].idxmax(), 'Elevation']}-{srtm_aec['z_score'].max()}-{srtm_aec['z_score'].idxmax()}.csv", index=False)

In [20]:
# extrapolated_aec.hvplot('CumArea', 'Elevation_Observed', kind='scatter') * \
# extrapolated_aec.hvplot('CumArea', 'Elevation')